   ### **pip指令**

In [ ]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


### 停用警告

In [ ]:
import warnings

# 禁用所有警告
warnings.filterwarnings('ignore')

### 下載股票數據

In [ ]:
import datetime
import yfinance as yf

stock_number = '0050.TW'
start = datetime.datetime(2000, 1, 1)

d = datetime.datetime(2024, 1, 16)

df = yf.download(stock_number, start=start, end=end)


### 數據處理

In [ ]:
import numpy as np

def Stock_Price_LSTM_Data_Preprocessing(df, mem_his_days, pre_days):
    df.dropna(inplace=True)
    df.sort_index(inplace=True)

    df['label'] = df['Close'].shift(-pre_days)

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    sca_X = scaler.fit_transform(df.iloc[:, :-1])

    from collections import deque
    deq = deque(maxlen=mem_his_days)

    X = []
    for i in sca_X:
        deq.append(list(i))
        if len(deq) == mem_his_days:
            X.append(list(deq))

    X_lately = X[-pre_days:]
    X = X[:-pre_days]

    y = df['label'].values[mem_his_days-1:-pre_days]


    X = np.array(X)
    y = np.array(y)

    return X, y, X_lately

X, y, X_lately = Stock_Price_LSTM_Data_Preprocessing(df, 5, 10)

print(len(X))
print(len(y))
print(len(X_lately))

### 模型超參數調整&訓練


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

lr_schedule = ExponentialDecay(initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.9)

pre_days = [5, 10, 15, 20, 25, 30]
mem_days = [5, 10, 15, 20, 25, 30]
lstm_layers = [1, 2, 3, 4, 5]
dense_layers = [1, 2, 3, 4, 5]
units = [16, 32, 64]

for the_pre_days in pre_days:
    for the_mem_days in mem_days:
        for the_lstm_layers in lstm_layers:
            for the_dense_layers in dense_layers:
                for the_units in units:
                    filepath = f'./models/{{val_mape:.2f}}_{{epoch:02d}}_pre_{the_pre_days}_men_{the_mem_days}_lstm_{the_lstm_layers}_dense_{the_dense_layers}_unit_{the_units}.h5'

                    checkpoint = ModelCheckpoint(
                        filepath=filepath,
                        #save_weights_only初訓練設定true只保存值無法讀檔
                        save_weights_only=True,
                        monitor='val_mape',
                        mode='min',
                        save_best_only=True)

                    X, y, X_lately = Stock_Price_LSTM_Data_Preprocessing(df, the_mem_days, the_pre_days)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.1)

                    model = Sequential()
                    model.add(LSTM(the_units, input_shape=X.shape[1:], return_sequences=True))
                    model.add(BatchNormalization())
                    model.add(Activation('relu'))
                    model.add(Dropout(0.1))

                    for i in range(the_lstm_layers):
                        model.add(LSTM(the_units, return_sequences=True))
                        model.add(BatchNormalization())
                        model.add(Activation('relu'))
                        model.add(Dropout(0.1))

                    model.add(LSTM(the_units))
                    model.add(BatchNormalization())
                    model.add(Activation('relu'))
                    model.add(Dropout(0.1))

                    for i in range(the_dense_layers):
                        model.add(Dense(the_units))
                        model.add(BatchNormalization())
                        model.add(Activation('relu'))
                        model.add(Dropout(0.1))

                    model.add(Dense(1))

                    # 使用自定義的學習率
                    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mse', metrics=['mape'])

                    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint])

### 儲存模型


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

lr_schedule = ExponentialDecay(initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.9)

pre_days = [5]  
mem_days = [25]
lstm_layers = [2]
dense_layers = [5]
units = [64]

for the_pre_days in pre_days:
    for the_mem_days in mem_days:
        for the_lstm_layers in lstm_layers:
            for the_dense_layers in dense_layers:
                for the_units in units:
                    filepath = f'./models/{{val_mape:.2f}}_{{epoch:02d}}_pre_{the_pre_days}_men_{the_mem_days}_lstm_{the_lstm_layers}_dense_{the_dense_layers}_unit_{the_units}.h5'


                    checkpoint = ModelCheckpoint(
                        filepath=filepath,
                        #save_weights_only二次訓練設定false才能讀擋
                        save_weights_only=False,
                        monitor='val_mape',
                        mode='min',
                        save_best_only=True)

                    X, y, X_lately = Stock_Price_LSTM_Data_Preprocessing(df, the_mem_days, the_pre_days)   
                    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.1)

                    model = Sequential()
                    model.add(LSTM(the_units, input_shape=X.shape[1:], return_sequences=True))
                    model.add(BatchNormalization())
                    model.add(Activation('relu'))
                    model.add(Dropout(0.1))

                    for i in range(the_lstm_layers):
                        model.add(LSTM(the_units, return_sequences=True))
                        model.add(BatchNormalization())
                        model.add(Activation('relu'))
                        model.add(Dropout(0.1))

                    model.add(LSTM(the_units))
                    model.add(BatchNormalization())
                    model.add(Activation('relu'))
                    model.add(Dropout(0.1))

                    for i in range(the_dense_layers):
                        model.add(Dense(the_units))
                        model.add(BatchNormalization())
                        model.add(Activation('relu'))
                        model.add(Dropout(0.1))

                    model.add(Dense(1))

                    # 使用自定義的學習率
                    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mse', metrics=['mape'])

                    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint])


### 尋找模型檔案


In [ ]:
import os

# 確定當前工作目錄
current_directory = %pwd  # %pwd 這個是 Jupyter Notebook 內建的魔術指令，如果在其他 IDE 中需要替換成 os.getcwd()

# 指定檔案路徑
folder_path = os.path.join(current_directory, 'models')

# 列出該資料夾下的所有檔案
files = os.listdir(folder_path)

# 過濾出以 '.h5' 結尾的檔案
model_files = [file for file in files if file.endswith('.h5')]

# 輸出所有模型檔案
print("找到的模型檔案：")
for model_file in model_files:
    print(os.path.join(folder_path, model_file))



### 載入&評估模型

In [ ]:
from tensorflow.keras.models import load_model

# 提供最佳模型的檔案路徑
filepath = r'{best_data_filepath}'

# 定義任何自定義的層或函數
custom_objects = {
    # 如果有自定義的層或函數，請在這裡添加
}

# 載入模型
best_model = load_model(filepath, custom_objects=custom_objects)

# 顯示模型摘要
best_model.summary()

# 評估模型表現
best_model.evaluate(X_test, y_test)

# 預測結果
pre = best_model.predict(X_test)
print(len(pre))

### 輸出視覺化結果


In [ ]:
import matplotlib.pyplot as plt

# 繪製測試集與預測結果的對比圖
df_time = df.index[-len(y_test):]  # df_time 是與 y_test 相對應的時間軸
plt.plot(df_time, y_test, color='blue', label='price')
plt.plot(df_time, pre, color='purple', label='predict')
plt.legend()
plt.show()